**Brain Tumor Segmentation BRATS2021**

Loading classes for segmentation from Brats 2020:

-[1] https://www.kaggle.com/frlemarchand/brain-tumour-segmentation-in-mri-slices/

-[2] https://www.kaggle.com/arashmehrzadi/brain-tumor-segmentation-unet/output


***After testing the performance for segmentation the rsna data with [2], the accuracy is not working very well, so, we are trying to run the U-Net with Brats 2021 data, and finally doing the ensemble with 2020, and check again:

- https://www.kaggle.com/dschettler8845/how-to-load-basic-data-exploration

In [12]:
from keras.layers import concatenate
###Libraries and imports
import numpy as np
import math
import random
import os
import shutil
import gzip
import shutil
import glob
import gc
import cv2 as cv
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import tarfile
import PIL
import scipy.misc
import skimage
import nibabel as nib
import matplotlib.pyplot as plt

from PIL import Image
from tqdm import tqdm
from tensorflow import keras
from keras.models import Model, load_model
from keras.layers import Input ,BatchNormalization , Activation 
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.layers.pooling import MaxPooling2D

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers 
from keras import backend as K
from sklearn.model_selection import train_test_split
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage import data
from skimage.io import imread
from scipy import ndimage as ndi
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from glob import glob


DEVICE = "GPU"

In [13]:
if DEVICE == "TPU":
    print("connecting to TPU...")
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        print("Could not connect to TPU")
        tpu = None

    if tpu:
        try:
            print("initializing  TPU ...")
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print("TPU initialized")
        except _:
            print("failed to initialize TPU")
    else:
        DEVICE = "GPU"

if DEVICE != "TPU":
    print("Using default strategy for CPU and single GPU")
    strategy = tf.distribute.get_strategy()

if DEVICE == "GPU":
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    

AUTO     = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

Using default strategy for CPU and single GPU
Num GPUs Available:  0
REPLICAS: 1


In [14]:
def Convolution(input_tensor,filters):
    x = Conv2D(filters=filters,kernel_size=(3, 3),padding = 'same',strides=(1, 1))(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 
    return x

def model(input_shape):
    
    inputs = Input((input_shape))
    
    conv_1 = Convolution(inputs,32)
    maxp_1 = MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same') (conv_1)
    
    conv_2 = Convolution(maxp_1,64)
    maxp_2 = MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same') (conv_2)
    
    conv_3 = Convolution(maxp_2,128)
    maxp_3 = MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same') (conv_3)
    
    conv_4 = Convolution(maxp_3,256)
    maxp_4 = MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same') (conv_4)
    
    conv_5 = Convolution(maxp_4,512)
    upsample_6 = UpSampling2D((2, 2)) (conv_5)
    
    conv_6 = Convolution(upsample_6,256)
    upsample_7 = UpSampling2D((2, 2)) (conv_6)
    
    upsample_7 = concatenate([upsample_7, conv_3])
    
    conv_7 = Convolution(upsample_7,128)
    upsample_8 = UpSampling2D((2, 2)) (conv_7)
    
    conv_8 = Convolution(upsample_8,64)
    upsample_9 = UpSampling2D((2, 2)) (conv_8)
    
    upsample_9 = concatenate([upsample_9, conv_1])
    
    conv_9 = Convolution(upsample_9,32)
    outputs = Conv2D(1, (1, 1), activation='sigmoid') (conv_9)
    
    model = Model(inputs=[inputs], outputs=[outputs]) 
    
    return model

# Computing Dice_Coefficient
def dice_coef(y_true, y_pred, smooth=1.0):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Computing Precision 
def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

# Computing Sensitivity      
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

# Computing Specificity
def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

# Accuracy vs Epoch
def Accuracy_Graph(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    #plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.subplots_adjust(top=1.00, bottom=0.0, left=0.0, right=0.95, hspace=0.25,
                        wspace=0.35)
    plt.show()
    
# Dice Similarity Coefficient vs Epoch
def Dice_coefficient_Graph(history):

    plt.plot(history.history['dice_coef'])
    plt.plot(history.history['val_dice_coef'])
    #plt.title('Dice_Coefficient')
    plt.ylabel('Dice_Coefficient')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.subplots_adjust(top=1.00, bottom=0.0, left=0.0, right=0.95, hspace=0.25,
                        wspace=0.35)
    plt.show()
# Loss vs Epoch
def Loss_Graph(history):

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    #plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.subplots_adjust(top=1.00, bottom=0.0, left=0.0, right=0.95, hspace=0.25,
                        wspace=0.35)
    plt.show()

print("-- Compiling the model with: ", DEVICE)
input_size = 240
if DEVICE=='TPU':
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
    with tpu_strategy.scope():
        model = model(input_shape = (input_size, input_size, 1))
        Adam=optimizers.Adam(lr=0.001)
        model.compile(optimizer=Adam, loss='binary_crossentropy', metrics=['accuracy',dice_coef,precision,sensitivity,specificity])        
else:   
    model = model(input_shape = (input_size, input_size, 1))
    Adam=optimizers.Adam(lr=0.001)
    model.compile(optimizer=Adam, loss='binary_crossentropy', metrics=['accuracy',dice_coef,precision,sensitivity,specificity])

-- Compiling the model with:  GPU


In [15]:
!rm -Rf ./post_process_data
!rm -Rf ./data
!rm -Rf ./final_models

Der Befehl "rm" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.
Der Befehl "rm" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.
Der Befehl "rm" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


In [16]:
#!mkdir data

tar = tarfile.open("/kaggle/input/brats-2021-task1/BraTS2021_Training_Data.tar")
tar.extractall(".data")
tar.close()

tar = tarfile.open("/kaggle/input/brats-2021-task1/BraTS2021_00495.tar")
tar.extractall("./data")
tar.close()

tar = tarfile.open("/kaggle/input/brats-2021-task1/BraTS2021_00621.tar")
tar.extractall("./data", )
tar.close()

img_id = "01281"
plt.figure(figsize=(18, 5))
for i, nii in enumerate([f'./data/BraTS2021_{img_id}/BraTS2021_{img_id}_{s_type}.nii.gz' for s_type in ["flair", "t1", "t1ce", "t2", "seg"]]):
    plt.subplot(1,5,i+1)
    image=nib.load(nii).get_fdata()
    plt.title(nii.rsplit("_", 1)[1].split(".", 1)[0], fontweight="bold")
    plt.axis(False)
    plt.imshow(image[:, :, 80], cmap="bone")
plt.tight_layout()    
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/brats-2021-task1/BraTS2021_Training_Data.tar'

In [ ]:
!ls ./data/BraTS2021_00000/

**Dataset organization in subforders**

In [ ]:
import glob
Path_brats= './data/BraTS2021_'
post_data='./post_process_data'
final_models='./final_models'
os.mkdir(post_data)
os.mkdir(final_models)

!ls ./data/ | awk -F'[_]' '{print $2}' > test.txt
post_data='./post_process_data/'
with open('./test.txt') as f:
    for linea in f:
        path_dest=post_data+linea
        path_dest=path_dest.replace('\n','')
        if os.path.isdir(path_dest) != True:
            os.mkdir(path_dest)
        create_path_brats=Path_brats+linea+'/*'
        for name in sorted(glob.glob(create_path_brats.replace('\n',''))):        
            shutil.move(name,path_dest)

**Preparing dataset and model**

In [ ]:
def Data_Concatenate(Input_Data):
    counter=0
    Output= []
    for i in range(5):
        print('$')
        c=0
        counter=0
        for ii in range(len(Input_Data)):
            if (counter != len(Input_Data)):
                a= Input_Data[counter][:,:,:,i]
                #print('a={}'.format(a.shape))
                b= Input_Data[counter+1][:,:,:,i]
                #print('b={}'.format(b.shape))
                if(counter==0):
                    c= np.concatenate((a, b), axis=0)
                    print('c1={}'.format(c.shape))
                    counter= counter+2
                else:
                    c1= np.concatenate((a, b), axis=0)
                    c= np.concatenate((c, c1), axis=0)
                    print('c2={}'.format(c.shape))
                    counter= counter+2
        c= c[:,:,:,np.newaxis]
        Output.append(c)
    return Output
    
def launch_model(Input_Data,code,model):
    
    InData= Data_Concatenate(Input_Data)
    AIO= concatenate(InData, axis=3)
    AIO=np.array(AIO,dtype='float32')
    TR=np.array(AIO[:,:,:,1],dtype='float32')
    TRL=np.array(AIO[:,:,:,4],dtype='float32')
    X_train , X_test, Y_train, Y_test = train_test_split(TR, TRL, test_size=0.15, random_state=32)
    AIO=TRL=0

    # Fitting the model over the data
    print("-- Fitting the model over the data --")
    history = model.fit(X_train,Y_train,batch_size=32,epochs=20,validation_split=0.20,verbose=1,initial_epoch=0)
    
    # Evaluating the model on the training and testing data 
    print("-- Evaluating the model on the training and testing data --")
    model.evaluate(x=X_train, y=Y_train, batch_size=32 , verbose=1, sample_weight=None, steps=None)
    model.evaluate(x=X_test, y=Y_test, batch_size=32, verbose=1, sample_weight=None, steps=None)    
    
    # Plotting the Graphs of Accuracy, Dice_coefficient, Loss at each epoch on Training and Testing data
    print("-- Plotting the Graphs of Accuracy, Dice_coefficient, Loss at each epoch on Training and Testing data --")
    Accuracy_Graph(history)
    Dice_coefficient_Graph(history)
    Loss_Graph(history)
               
    model.save('./final_models/BraTs2021_'+code+'.h5')

**Launch model**

* top_limit_number = Number of loops that we take.
* split_number = Number of elements that we take for the look
* init_counter = Loop local counter
* inside_split_countert = Loop global counter


In [ ]:
import glob
Path= './post_process_data'
p=os.listdir(Path)
Input_Data= []


def Data_Preprocessing(modalities_dir):
    all_modalities = []    
    for modality in modalities_dir:      
        nifti_file   = nib.load(modality)
        brain_numpy  = np.asarray(nifti_file.dataobj)    
        all_modalities.append(brain_numpy)
    brain_affine   = nifti_file.affine
    all_modalities = np.array(all_modalities)
    all_modalities = np.rint(all_modalities).astype(np.int16)
    all_modalities = all_modalities[:, :, :, :]
    all_modalities = np.transpose(all_modalities)
    return all_modalities

top_limit_number = 15
split_number = 20
init_counter=0
inside_split_countert=1
total_count_img, partial_count_img = len(p), int(len(p)/split_number)

for i in tqdm(p):
    if (int(init_counter*inside_split_countert) == total_count_img) or (int(top_limit_number*split_number)==int(init_counter*inside_split_countert)):
        print("Launch Final model.")
        launch_model(Input_Data,str(init_counter), model)
        del(Input_Data)
        gc.collect()
        break    
    if (init_counter == split_number):
        print("Launch model :"+str(init_counter*inside_split_countert))
        launch_model(Input_Data,str(init_counter*inside_split_countert),model)
        del(Input_Data)
        gc.collect()
        Input_Data= []
        inside_split_countert=inside_split_countert+1
        init_counter=0
    create_path_post=Path+'/'+i+'/*'
    
    for name in sorted(glob.glob(create_path_post.replace('\n',''))):
        os.system('gunzip ' + name)
    brain_dir = os.path.normpath(Path+'/'+i+'/')
    flair     = glob.glob(os.path.join(brain_dir, '*_flair*.nii'))
    t1        = glob.glob(os.path.join(brain_dir, '*_t1*.nii'))
    t1ce      = glob.glob(os.path.join(brain_dir, '*_t1ce*.nii'))
    t2        = glob.glob(os.path.join(brain_dir, '*_t2*.nii'))
    gt        = glob.glob( os.path.join(brain_dir, '*_seg*.nii'))
    modalities_dir = [flair[0], t1[0], t1ce[0], t2[0], gt[0]]
    P_Data = Data_Preprocessing(modalities_dir)
    Input_Data.append(P_Data)
    shutil.rmtree(Path+'/'+i)
    init_counter = init_counter + 1
